# Downsampling of large image data

I used this Notebook to create smaller versions of the 2GB Mars elevation map
provided by the USGS (which can be found [here](Mars Map)). I found that
`scipy.misc.imresize()` required far too much RAM (exceeding the available
physical 16GB RAM of my machine).

This method uses two-pass bilinear downscaling. Note that clearly visibly a lot
of detail is lost by the x8 and x16 downsampling.

[Mars Map]: https://astrogeology.usgs.gov/search/map/Mars/GlobalSurveyor/MOLA/Mars_MGS_MOLA_DEM_mosaic_global_463m

In [33]:
img = tifffile.imread('./Mars_MGS_MOLA_DEM_mosaic_global_463m.tif')

In [34]:
import numpy as np
import tifffile


def resize(img, size):
  dtype = img.dtype

  xin = np.linspace(0, 1, img.shape[1])
  xot = np.linspace(0, 1, size[1])
  img = np.array([np.interp(xot, xin, col) for col in img])

  xin = np.linspace(0, 1, img.shape[0])
  xot = np.linspace(0, 1, size[0])
  img = np.array([np.interp(xot, xin, col) for col in img.T]).T
  
  return img.astype(dtype)


for i in (2, 4, 8, 16):
  new_image = resize(img, (img.shape[0] // i, img.shape[1] // i))
  tifffile.imsave('./Mars_MGS_MOLA_DEM_mosaic_global_463m_x{}.tif'.format(i), new_image)